In [1]:
import sys
sys.path.insert(0, "..")

In [2]:
import medspacy
from medspacy.ner import TargetRule

# Overview
Being able to visualize the results of an NLP model are extremely useful for both development/debugging and sharing results. MedspaCy provides visualization capabilties based on spaCy's awesome [displaCy](https://spacy.io/usage/visualizers) module. MedspaCy's visualizers are implemented in `medspacy.visualization`.

There are 2 **flavors** of visualization for medspaCy:
- **ent**: Highlight spans of text in the document corresponding to the ents, context modifiers, and section titles
- **dep**: Show relationships between entities and modifying context spans

First, let's set up some examples:

In [3]:
texts = [
    "Family History: Mother with stroke at age 82.",
    "Past Medical History: colon cancer",
    "Allergies: Hydrochlorothiazide",
    "Some metastasis.",
    "Patient presents for radiotherapy to treat her breast cancer.",
    
]

In [15]:
nlp = medspacy.load(enable=['pyrush', 'target_matcher', 'context', 'sectionizer'])

In [16]:
nlp.pipe_names

['medspacy_pyrush',
 'medspacy_target_matcher',
 'medspacy_context',
 'medspacy_sectionizer']

In [17]:
target_rules = [
    TargetRule(literal="abdominal pain", category="PROBLEM"),
    TargetRule("stroke", "PROBLEM"),
    TargetRule("hemicolectomy", "TREATMENT"),
    TargetRule("Hydrochlorothiazide", "TREATMENT"),
    TargetRule("colon cancer", "PROBLEM"),
    TargetRule("breast cancer", "PROBLEM"),
    TargetRule("radiotherapy", "TREATMENT"),
    TargetRule("metastasis", "PROBLEM"),
    
]

In [18]:
nlp.get_pipe("medspacy_target_matcher").add(target_rules)

In [19]:
docs = list(nlp.pipe(texts))

# I. `visualize_ent`
The `visualize_ent` function displays the entites, context modifiers, and section titles. By default, a color cycle is generated for entities and modifiers, and the entity/modifier class is displayed next to the text. Section titles are shown in gray with the normalized section title in **<<>>**.

In [20]:
from medspacy.visualization import visualize_ent

In [21]:
visualize_ent(docs[0])

In [22]:
visualize_ent(docs[1])

You can choose whether or not to highlight modifiers and section titles:

In [23]:
visualize_ent(docs[0], context=False, sections=False)

You can also pass in a class name -> color mapping in the `colors` argument:

In [24]:
visualize_ent(docs[0], colors={"HISTORICAL": "yellow", "FAMILY": "red", "PROBLEM": "turquoise"})

And instead of displaying inline you can also return an HTML by passing in `jupyter=False`:

In [25]:
html = visualize_ent(docs[0], jupyter=False)

In [26]:
print(html)

<div class="entities" style="line-height: 2.5; direction: ltr">
<mark class="entity" style="background: #dee0e3; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">
    Family History:
    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; vertical-align: middle; margin-left: 0.5rem"><< FAMILY_HISTORY >></span>
</mark>

<mark class="entity" style="background: #ff7f0e; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">
    History
    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; vertical-align: middle; margin-left: 0.5rem">HISTORICAL</span>
</mark>
: 
<mark class="entity" style="background: #2ca02c; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">
    Mother
    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; vertical-align: middle; margin-left: 0.5rem">FAMILY</span>
</mark

# `visualize_dep`
The function `visualize_dep` shows the relationships between context modifiers and entities. This can be useful for demonstrating the linguistic relationships in the sentence and also for debugging attributes like negation and family history.

In [27]:
from medspacy.visualization import visualize_dep

In [28]:
visualize_dep(docs[0])

**Note**: `visualize_dep` works best for small snippets of text, typically a single sentence (which is usually the scope for context anyway). Rather than visualizing an entire clinical document, it's recommend that you visualize only a single entity:
```python
visualize_dep(nlp(ent.sent.text))
```

Similar to `visualize_ent`, you can also return the html string:

In [29]:
html = visualize_dep(docs[0], jupyter=False)

# IPywidget 
MedspaCy also comes with an IPywidget which allows you to scroll through multiple docs and toggle between `ent`, `dep`, or combined views. To use the widget, you'll need to make sure you've installed and enabled Ipywidgets in Jupyter: https://ipywidgets.readthedocs.io/en/latest/

To instantiate the widget, pass in your list of docs. Then scroll through the list view the different output options.

In [30]:
from medspacy.visualization import MedspaCyVisualizerWidget

In [31]:
widget = MedspaCyVisualizerWidget(docs)

Box(children=(HBox(children=(RadioButtons(options=('Ent', 'Dep', 'Both'), value='Ent'), Button(description='Pr…